In [ ]:
# start spark stream by running ./scripts/run-spark-stream.sh locally 

In [ ]:
import sys
from session import create_spark_session
from create_topic import main
import datetime
from kafka import KafkaConsumer

spark = create_spark_session(app_name='kafka-stream')

In [ ]:
# Create a SparkSession
jars = ["org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.4"]
#    .config("spark.jars", "/opt/spark/jars/kafka-clients-3.4.1.jar") \
# Set up the Kafka configuration
BOOTSTRAP_SERVERS = "kafka-broker:9092"
topic = 'message'

In [ ]:
# Check what catalog is currently active
print(f"Current catalog: {spark.catalog.currentCatalog()}")

# List all available catalogs
print("Available catalogs:")
spark.catalog.listCatalogs()

# List all databases in the current catalog
print("Databases in current catalog:")
spark.catalog.listDatabases()

spark.sql("SHOW NAMESPACES;").show()

In [ ]:
# read from iceberg table using spark
spark.sql("SELECT * FROM iceberg.spark.message LIMIT 10").show()

In [ ]:
spark.sql("SHOW TABLES in spark;").show()

In [ ]:

topic_name = 'message'
consumer = KafkaConsumer(
    topic,
    bootstrap_servers=BOOTSTRAP_SERVERS,
    auto_offset_reset="earliest",  # Start from the beginning
    enable_auto_commit=False,
    group_id="python-consumer-group",
    value_deserializer=lambda x: x.decode("utf-8"),  # Decode bytes to string
)

print("✅ Successfully connected to Kafka!")
print(f"📋 Consumer group: python-consumer-group")
print(f"🔄 Auto offset reset: earliest")
print("=" * 50)


In [ ]:
message_count = 0

max_messages = 10
# Consume messages
for message in consumer:
    message_count += 1

    print(f"📨 Message #{message_count}")
    print(f"   Topic: {message.topic}")
    print(f"   Partition: {message.partition}")
    print(f"   Offset: {message.offset}")
    print(f"   Key: {message.key}")
    print(f"   Value: {message.value}")
    print("-" * 30)

    # Stop after max_messages if specified
    if max_messages and message_count >= max_messages:
        print(f"✅ Consumed {message_count} messages. Stopping...")
        break